In [2]:
import boto3
import pandas as pd
import numpy as np
import json
import gzip
import importlib
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from boto3.dynamodb.types import Binary
from importlib import reload
from dynamodb import DynamoTable

In [3]:
dt = DynamoTable()

In [4]:
dt.create_table(table_name="dragon_stats", 
                partition_key="dragon_name", 
                partition_key_type="S", 
                provisioned=False
)

dt.load_json("dragons_dynamo/dragon_stats_one.json")
dt.load_json("dragons_dynamo/dragon_stats_two.json")

In [ ]:
try: 
    dt.create_table("data_name", "Inv_ID", "S", "Bill_ID", "S")
except:
    dt.select_table("data_name")

In [ ]:
list(dt.dyn_resource.tables.all())

In [ ]:
s_out = """Offer for a Basic Digital Access subscription is not open to current digital news s
ubscribers. Your payment method will automatically be charged in advance the introductory 
offer rate every 4 weeks for 1 year, and after 1 year the standard rate every 4 weeks. 
Your subscription will continue until you cancel. Cancellation takes effect at the end 4
of your current billing period. The print edition of The New York Times, Games, Cooking or 
Wirecutter are not included. Taxes may apply. Offer terms are subject to change.
"""
s_out2 = """At its fifth hearing on the Capitol riot, the panel plans to show evidence about how Donald Trump tried to manipulate the Justice Department to stay in power.
The committee is presenting his actions as a critical strand in a multilayered effort by the former president to subvert the election. Watch the hearing live.
"""
s_out3 = "At itsid Watch the hearing live."

item1 = {
    "Inv_ID": "Invoice-40539",
    "Bill_ID": "Bill-235",
    "Data1": s_out,
    "Data2": s_out2
}

item3 = {
    "Inv_ID": "Invoice-40539",
    "Bill_ID": "Bill-15",
    "Data2": s_out3
}

df = pd.DataFrame([item1, item3], index=range(2))

In [ ]:
import dynamodb
importlib.reload(dynamodb)
from dynamodb import DynamoTable
dt = DynamoTable()
dt.select_table("data_name")

In [ ]:
dt.batch_pandas(df, compress=True)

In [ ]:
dt.query_items("Invoice-40539", to_pandas=True)

In [ ]:
dt.select_table("dragons")

In [22]:
dt.dyn_client.describe_table(TableName="dragons")

{'Table': {'AttributeDefinitions': [{'AttributeName': 'dragon_name',
    'AttributeType': 'S'}],
  'TableName': 'dragons',
  'KeySchema': [{'AttributeName': 'dragon_name', 'KeyType': 'HASH'}],
  'TableStatus': 'ACTIVE',
  'CreationDateTime': datetime.datetime(2022, 6, 24, 20, 22, 25, 751000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:210229652268:table/dragons',
  'TableId': '75f1a5c0-a805-4d43-8bed-36066451c86f',
  'BillingModeSummary': {'BillingMode': 'PAY_PER_REQUEST',
   'LastUpdateToPayPerRequestDateTime': datetime.datetime(2022, 6, 24, 20, 22, 25, 751000, tzinfo=tzlocal())}},
 'ResponseMetadata': {'RequestId': '1I60UJBKKFHM6J5BDUDG2C841BVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sat, 25 Jun 2022 00:12:15 GMT',
   'content-type': 'application/x-amz-jso

In [14]:
dt

- Table name: dragon_stats            
- Table arn: arn:aws:dynamodb:us-east-1:254306263884:table/dragon_stats            
- Table creation: 2022-06-25 00:28:28.276000+00:00            
- [{'AttributeName': 'dragon_name', 'KeyType': 'HASH'}]            
- [{'AttributeName': 'dragon_name', 'AttributeType': 'S'}]

In [8]:
import boto3

# Boto3 is the AWS SDK library for Python.
# You can use the low-level client to make API calls to DynamoDB.
client = boto3.client('dynamodb', region_name='us-east-1')

try:
    resp = client.update_table(
        TableName="dragon_stats",
        # Any attributes used in your new global secondary index must be declared in AttributeDefinitions
        AttributeDefinitions=[
            {
                "AttributeName": "family",
                "AttributeType": "S"
            },
        ],
        # This is where you add, update, or delete any global secondary indexes on your table.
        GlobalSecondaryIndexUpdates=[
            {
                "Create": {
                    # You need to name your index and specifically refer to it when using it for queries.
                    "IndexName": "CategoryIndex",
                    # Like the table itself, you need to specify the key schema for an index.
                    # For a global secondary index, you can use a simple or composite key schema.
                    "KeySchema": [
                        {
                            "AttributeName": "family",
                            "KeyType": "HASH"
                        }
                    ],
                    # You can choose to copy only specific attributes from the original item into the index.
                    # You might want to copy only a few attributes to save space.
                    "Projection": {
                        "ProjectionType": "ALL"
                    },
                    # Global secondary indexes have read and write capacity separate from the underlying table.
                    
                }
            }
        ],
    )
    print("Secondary index added!")
except Exception as e:
    print("Error updating table:")
    print(e)

Secondary index added!


In [20]:
dt.table.global_secondary_indexes

[{'IndexName': 'CategoryIndex',
  'KeySchema': [{'AttributeName': 'family', 'KeyType': 'HASH'}],
  'Projection': {'ProjectionType': 'ALL'},
  'IndexStatus': 'ACTIVE',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 0,
   'WriteCapacityUnits': 0},
  'IndexSizeBytes': 0,
  'ItemCount': 0,
  'IndexArn': 'arn:aws:dynamodb:us-east-1:254306263884:table/dragon_stats/index/CategoryIndex'}]

In [17]:
dt.table.reload()

In [23]:
dt.table.query(IndexName="CategoryIndex",
    KeyConditionExpression=Key('family').eq('red')
              )

{'Items': [{'location_neighborhood': 'shank rd',
   'damage': Decimal('8'),
   'location_city': 'dover',
   'family': 'red',
   'description': 'Isilier is one of the noblest dragon generals in the fire army. His fire attack is one of the strongest in the fleet.',
   'protection': Decimal('7'),
   'location_country': 'usa',
   'location_state': 'delaware',
   'dragon_name': 'Isilier'},
  {'location_neighborhood': 'margate st',
   'damage': Decimal('4'),
   'location_city': 'encino',
   'family': 'red',
   'description': 'Norslo breathes lava instead of fire. He creates magna flows and traps his enemies.',
   'protection': Decimal('7'),
   'location_country': 'usa',
   'location_state': 'california',
   'dragon_name': 'Norslo'},
  {'location_neighborhood': 'edna ave',
   'damage': Decimal('9'),
   'location_city': 'las vegas',
   'family': 'red',
   'description': 'Crimson is a soldier in the dragon fire army. He breathes fire lightning for a multiple attack.',
   'protection': Decimal('